#Importing libraries

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#Data Preprocessing


###Traning Image preprocessing

In [3]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'fruit-and-vegetable-image-recognition/train',
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = None,
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (64,64),
    shuffle = True,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = 'bilinear',
    follow_links = False,
    crop_to_aspect_ratio = False
)

Found 3115 files belonging to 36 classes.


###validation image preprocessing

In [4]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'fruit-and-vegetable-image-recognition/validation',
    labels = 'inferred',
    label_mode = 'categorical',
    class_names = None,
    color_mode = 'rgb',
    batch_size = 32,
    image_size = (64,64),
    shuffle = True,
    seed = None,
    validation_split = None,
    subset = None,
    interpolation = 'bilinear',
    follow_links = False,
    crop_to_aspect_ratio = False
)

Found 351 files belonging to 36 classes.


#Building Model

In [5]:
with tf.device('/device:GPU:0'):
     cnn = tf.keras.models.Sequential()
     ###Building Convolution Layer
     cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu',input_shape=(64,64,3)))
     cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
     cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
     cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
     cnn.add(tf.keras.layers.Dropout(0.5)) #To avoid overfitting
     cnn.add(tf.keras.layers.Flatten())
     cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))
     cnn.add(tf.keras.layers.Dense(units=36,activation='softmax'))
     ##Compiling and Training Phase
     cnn.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
     training_history = cnn.fit(x=training_set,validation_data=validation_set,epochs=30)

Epoch 1/30
98/98 [==============================] - 19s 153ms/step - loss: 7.1219 - accuracy: 0.0501 - val_loss: 3.3481 - val_accuracy: 0.1225
Epoch 2/30
98/98 [==============================] - 16s 152ms/step - loss: 5.1205 - accuracy: 0.0770 - val_loss: 7.4000 - val_accuracy: 0.0883
Epoch 3/30
98/98 [==============================] - 17s 157ms/step - loss: 3.8872 - accuracy: 0.1274 - val_loss: 2.8758 - val_accuracy: 0.2735
Epoch 4/30
98/98 [==============================] - 17s 157ms/step - loss: 3.9024 - accuracy: 0.1547 - val_loss: 2.5898 - val_accuracy: 0.4359
Epoch 5/30
98/98 [==============================] - 17s 160ms/step - loss: 3.5151 - accuracy: 0.1968 - val_loss: 2.2634 - val_accuracy: 0.4587
Epoch 6/30
98/98 [==============================] - 17s 158ms/step - loss: 3.2191 - accuracy: 0.2565 - val_loss: 1.9439 - val_accuracy: 0.5014
Epoch 7/30
98/98 [==============================] - 17s 157ms/step - loss: 2.6509 - accuracy: 0.3210 - val_loss: 2.5307 - val_accuracy: 0.3020